<a href="https://colab.research.google.com/github/swaroop-2007/ds_salary_prediction/blob/main/ML_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [54]:
df = pd.read_csv('https://github.com/swaroop-2007/ds_salary_prediction/blob/main/Cleaned_data.csv?raw=true')
df.tail()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,...,maxsalary,averagesalary,Company,state,same_state,age,python y/n,spark y/n,excel y/n,aws y/n
737,"Sr Scientist, Immuno-Oncology - Oncology",$58K-$111K (Glassdoor est.),Site Name: USA - Massachusetts - Cambridge\nPo...,3.9,GSK\n3.9,"Cambridge, MA","Brentford, United Kingdom",10000+ employees,1830,Company - Public,...,111,84.5,GSK\n,MA,0,192,0,0,0,1
738,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrite\n4.4,"Nashville, TN","San Francisco, CA",1001 to 5000 employees,2006,Company - Public,...,133,102.5,Eventbrite\n,TN,0,16,1,1,0,1
739,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute\n2.6,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000 employees,1984,College / University,...,91,73.5,Software Engineering Institute\n,PA,1,38,0,0,1,0
740,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LLC\n3.2","Allentown, PA","Chadds Ford, PA",1 to 50 employees,-1,Company - Private,...,160,127.5,"Numeric, LLC\n",PA,0,-1,0,0,1,0
741,Research Scientist – Security and Privacy,$61K-$126K (Glassdoor est.),Returning Candidate? Log back in to the Career...,3.6,Riverside Research Institute\n3.6,"Beavercreek, OH","Arlington, VA",501 to 1000 employees,1967,Nonprofit Organization,...,126,93.5,Riverside Research Institute\n,OH,0,55,1,0,0,0


In [55]:
df['Competitors']
df['numberofCompetitors'] = df['Competitors'].apply(lambda x: len(x.split(','))if x != '-1' else 0)

In [56]:
df['lengthOfDescription'] = df['Job Description'].apply(lambda x: len(x))
df['lengthOfDescription']

0      2536
1      4783
2      3461
3      3883
4      2728
       ... 
737    6162
738    6130
739    3078
740    1642
741    3673
Name: lengthOfDescription, Length: 742, dtype: int64

In [57]:
def title_simplifier(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'analyst' in title.lower():
        return 'analyst'
    elif 'machine learning' in title.lower():
        return 'mle'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    else:
        return 'na'
    
def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'sr' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
            return 'senior'

In [58]:
df['jobsimp'] = df['Job Title'].apply(title_simplifier)
df.jobsimp.value_counts()

data scientist    279
na                184
data engineer     119
analyst           102
manager            22
mle                22
director           14
Name: jobsimp, dtype: int64

In [59]:
df['seniority'] = df['Job Title'].apply(seniority)
df.seniority.value_counts()

senior    220
Name: seniority, dtype: int64

In [60]:
df['state'] = df['state'].apply(lambda x: x.strip() if x.strip().lower()!= 'los angeles' else 'CA')
df['state']

0      NM
1      MD
2      FL
3      WA
4      NY
       ..
737    MA
738    TN
739    PA
740    PA
741    OH
Name: state, Length: 742, dtype: object

In [61]:
df['Company']
#Removing "\n from Name of Companies"
df['Company'] = df['Company'].apply(lambda x: x.replace('\n', ''))

In [62]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hoursalary', 'employeerprovided', 'minsalary', 'maxsalary',
       'averagesalary', 'Company', 'state', 'same_state', 'age', 'python y/n',
       'spark y/n', 'excel y/n', 'aws y/n', 'numberofCompetitors',
       'lengthOfDescription', 'jobsimp', 'seniority'],
      dtype='object')

### From the columns we will take Average Salary, Size, Type of Ownership, Rating, Industry, Sector, Number of Competitors, Revenue, Hour Salary, Employeer Provided Salary, State of the Job, Python, Age, Same state as Headquarters, Excel, AWS, Job Simp, Seniority, Length of Description for the ML model building purpose. 

In [63]:
df_model = df[['averagesalary','Rating','Size','Type of ownership','Industry','Sector','Revenue','numberofCompetitors','hoursalary','employeerprovided',
             'state','same_state','age','python y/n','aws y/n','excel y/n','jobsimp','seniority','lengthOfDescription']]

In [64]:
df_model.head()

,averagesalary,Rating,Size,Type of ownership,Industry,Sector,Revenue,numberofCompetitors,hoursalary,employeerprovided,state,same_state,age,python y/n,aws y/n,excel y/n,jobsimp,seniority,lengthOfDescription
0,72.0,3.8,501 to 1000 employees,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),0,0,0,NM,0,49,1,0,1,data scientist,None,2536
1,87.5,3.4,10000+ employees,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),0,0,0,MD,0,38,1,0,0,data scientist,None,4783
2,85.0,4.8,501 to 1000 employees,Company - Private,Security Services,Business Services,$100 to $500 million (USD),0,0,0,FL,1,12,1,0,1,data scientist,None,3461
3,76.5,3.8,1001 to 5000 employees,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),3,0,0,WA,1,57,1,0,0,data scientist,None,3883
4,114.5,2.9,51 to 200 employees,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,3,0,0,NY,1,24,1,0,1,data scientist,None,2728


### Getting Dummy data for our model DataFrame

In [65]:
dummy = pd.get_dummies(df_model)

In [66]:
dummy

,averagesalary,Rating,numberofCompetitors,hoursalary,employeerprovided,same_state,age,python y/n,aws y/n,excel y/n,...,state_WA,state_WI,jobsimp_analyst,jobsimp_data engineer,jobsimp_data scientist,jobsimp_director,jobsimp_manager,jobsimp_mle,jobsimp_na,seniority_senior
0,72.0,3.8,0,0,0,0,49,1,0,1,...,0,0,0,0,1,0,0,0,0,0
1,87.5,3.4,0,0,0,0,38,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,85.0,4.8,0,0,0,1,12,1,0,1,...,0,0,0,0,1,0,0,0,0,0
3,76.5,3.8,3,0,0,1,57,1,0,0,...,1,0,0,0,1,0,0,0,0,0
4,114.5,2.9,3,0,0,1,24,1,0,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,84.5,3.9,3,0,0,0,192,0,1,0,...,0,0,0,0,0,0,0,0,1,1
738,102.5,4.4,3,0,0,0,16,1,1,0,...,0,0,0,1,0,0,0,0,0,1
739,73.5,2.6,0,0,0,1,38,0,0,1,...,0,0,0,0,0,0,0,0,1,0
740,127.5,3.2,0,0,0,0,-1,0,0,1,...,0,0,0,0,0,0,1,0,0,0


### Train-Test Split 

In [67]:
from sklearn.model_selection import train_test_split

In [68]:
x = dummy.drop('averagesalary', axis=1)
y = dummy.averagesalary.values

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


### Multiple Linear Regression Model

In [69]:
import statsmodels.api as sm

In [70]:
X_sm = sm.add_constant(x)
model = sm.OLS(y, X_sm)
model.fit().summary()

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.708
Model:                            OLS   Adj. R-squared:                  0.639
Method:                 Least Squares   F-statistic:                     10.39
Date:                Wed, 21 Dec 2022   Prob (F-statistic):           1.63e-95
Time:                        05:53:50   Log-Likelihood:                -3311.8
No. Observations:                 742   AIC:                             6906.
Df Residuals:                     601   BIC:                             7556.
Df Model:                         140                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
const                                                45.1983      6.767      6.679      0.000      31.908      58.488
Rating                                                3.6867      1.926      1.914      0.056      -0.095       7.469
numberofCompetitors                                   2.2363      0.866      2.581      0.010       0.535       3.938
hoursalary                                          -36.6772      9.129     -4.018      0.000     -54.606     -18.749
employeerprovided                                    31.7928     11.478      2.770      0.006       9.252      54.334
same_state                                            2.4896      2.612      0.953      0.341      -2.641       7.620
age                                                   0.0349      0.030      1.163      0.245      -0.024       0.094
python y/n                                            8.0711      2.491      3.240      0.001       3.179      12.964
aws y/n                                               1.0087      2.696      0.374      0.708      -4.287       6.304
excel y/n                                            -0.3247      2.259     -0.144      0.886      -4.762       4.113
lengthOfDescription                                  -0.0001      0.001     -0.185      0.853      -0.002       0.001
Size_-1                                              13.6822      8.637      1.584      0.114      -3.280      30.645
Size_1 to 50 employees                                1.1971      6.472      0.185      0.853     -11.513      13.907
Size_10000+ employees                                -2.5537      5.756     -0.444      0.657     -13.858       8.751
Size_1001 to 5000 employees                           6.3387      3.833      1.654      0.099      -1.188      13.866
Size_201 to 500 employees                             2.5226      3.853      0.655      0.513      -5.044      10.089
Size_5001 to 10000 employees                         -3.6745      5.408     -0.679      0.497     -14.296       6.947
Size_501 to 1000 employees                            2.4138      4.027      0.599      0.549      -5.496      10.323
Size_51 to 200 employees                              4.3078      4.400      0.979      0.328      -4.334      12.949
Size_Unknown                                         20.9644     14.002      1.497      0.135      -6.533      48.462
Type of ownership_-1                                 13.6822      8.637      1.584      0.114      -3.280      30.645
Type of ownership_College / University                6.5403     17.692      0.370      0.712     -28.205      41.286
Type of ownership_Company - Private                   8.8189      5.192      1.699      0.090      -1.377      19.015
Type of ownersh

### Model Explains 70% variation. 
### Depending on p value we can find which variables are relevant. eg: Python is relevant for the position. We want p value less than 0.05

In [71]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [72]:
lm = LinearRegression()
lm.fit(X_train, y_train)

LinearRegression()

In [73]:
np.mean(cross_val_score(lm,X_train,y_train, scoring = 'neg_mean_absolute_error', cv= 3))


-229514681.370401

### Cross value will take sample data, run the model on that sample. 

### Random Forest

In [74]:
from sklearn.ensemble import RandomForestRegressor

In [75]:
rf = RandomForestRegressor()

In [76]:
np.mean(cross_val_score(rf,X_train,y_train,scoring = 'neg_mean_absolute_error', cv= 3))

-15.073298509630996

### Better Values than regression

### Tuning

In [77]:
from sklearn.model_selection import GridSearchCV

In [78]:
parameters = {'n_estimators':range(10,300,10), 'criterion':('squared_error','absolute_error'), 'max_features':('auto','sqrt','log2')}

### Grid Search

In [79]:
gs = GridSearchCV(rf,parameters,scoring='neg_mean_absolute_error',cv=3)
gs.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'criterion': ('squared_error', 'absolute_error'),
                         'max_features': ('auto', 'sqrt', 'log2'),
                         'n_estimators': range(10, 300, 10)},
             scoring='neg_mean_absolute_error')

In [83]:
gs.best_score_
#

-14.776941187220373

In [84]:
gs.best_estimator_

RandomForestRegressor(n_estimators=130)

### 130 was the best no of estimators

### Testing ensemble models

In [85]:
tpred_lm = lm.predict(X_test)


In [86]:
tpred_rf = gs.best_estimator_.predict(X_test)

In [87]:
from sklearn.metrics import mean_absolute_error

In [88]:
mean_absolute_error(y_test,tpred_lm)



12426823.381992007

In [89]:

mean_absolute_error(y_test,tpred_rf)

10.964584408879713

### Random Forest improved the most